In [92]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import time
from pathlib import Path
import os
import openpyxl
from sec_cik_mapper import StockMapper
from datetime import datetime, timedelta
from urllib.parse import urljoin


In [72]:
# Get the current working directory
current_dir = os.getcwd()

# Resolve the parent directories
project_general_path = Path(current_dir).resolve()
print(project_general_path)

C:\Users\Adam Krupa\OneDrive\Pulpit\Investing\earnings-report-analysis


In [73]:
def get_tickers():
    # URL of the website containing the S&P 500 tickers
    url = 'https://www.slickcharts.com/sp500'

    # Fetch the page content with headers to avoid being blocked
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {'class': 'table table-hover table-borderless table-sm'})
        tickers = []
        
        # Extract tickers from the table
        if table:
            for row in table.find('tbody').find_all('tr'):
                columns = row.find_all('td')
                ticker = columns[2].text.strip()  # The third column contains the ticker symbol
                tickers.append(ticker)
        else:
            print("Table not found in the page")
    else:
        print("Failed to fetch S&P 500 tickers")
        tickers = []

    return tickers
    


In [86]:
tickers = get_tickers()

In [87]:
tickers_short = tickers[:10]

In [77]:
def get_dates(tickers):
    # Initialize an empty DataFrame to store valid earnings data
    data = pd.DataFrame(columns=['Ticker', 'Date'])

    # Loop through each stock ticker
    for ticker_symbol in tickers:
        ticker = yf.Ticker(ticker_symbol)
        # Fetch the earnings history (past earnings reports)
        earnings_history = ticker.earnings_dates
        
        # Skip if no earnings data is available
        if earnings_history is None:
            print(f"{ticker_symbol}: possibly delisted; no earnings dates found")
            continue

        # Iterate through each earnings report and determine the correct date classification
        for index, row in earnings_history.iterrows():
            report_time = pd.to_datetime(row.name)  # Access the index (which is the earnings date) and convert it to datetime
            # if report_time.hour >= 16:  # After 4 PM, classify as next day because the trade based on this knowledge can effectively only be executed the day after
            #     adjusted_date = (report_time + pd.Timedelta(days=1)).date()
            # else:  # Otherwise, use the same day
            #     adjusted_date = report_time.date()

            # Add each adjusted earnings date as a separate row in the DataFrame
            new_row = {'Ticker': ticker_symbol, 'Date': report_time}
            data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)

    return data


In [78]:
def get_cik(data):
    mapper = StockMapper()
    ticker_to_cik = mapper.ticker_to_cik

    # Ensure tickers are in uppercase to match the mapping keys
    data['Ticker'] = data['Ticker'].str.upper()

    # Map the 'Ticker' column to CIK numbers
    data['CIK'] = data['Ticker'].map(ticker_to_cik)

    return data


In [ ]:
def get_cik_n_names(data):
    # URL of the SEC's Company Tickers JSON file
    url = 'https://www.sec.gov/files/company_tickers.json'
    
    # Define headers with a User-Agent
    headers = {
        'User-Agent': 'Name Surname (email@gmail.com)'  # Replace with your actual name and email
    }
    
    try:
        # Fetch the JSON file with headers
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error for bad status codes
        
        # Parse the JSON data
        company_data = response.json()
        
        # Create mappings
        ticker_to_cik = {}
        ticker_to_name = {}
        
        for company in company_data.values():
            ticker = company['ticker'].upper()
            cik = str(company['cik_str']).zfill(10)
            name = company['title']
            ticker_to_cik[ticker] = cik
            ticker_to_name[ticker] = name
        
        # Ensure tickers are in uppercase to match the mapping keys
        data['Ticker'] = data['Ticker'].str.upper()
        
        # Map the 'Ticker' column to CIK numbers and company names
        data['CIK'] = data['Ticker'].map(ticker_to_cik)
        data['Name'] = data['Ticker'].map(ticker_to_name)
        
        return data
    
    except requests.RequestException as e:
        print(f'Error fetching data: {e}')
        return data

In [96]:
def get_dates_v2(data):
    # Ensure the input DataFrame has the required 'CIK' column
    if 'CIK' not in data.columns:
        raise ValueError("Input DataFrame must contain a 'CIK' column")

    # Initialize an empty DataFrame to store valid earnings data
    result_data = pd.DataFrame(columns=['CIK', 'Ticker', 'Date'])

    # Define a function to fetch filing dates for 10-K and 10-Q forms
    def fetch_filing_dates(cik):
        url = f'https://data.sec.gov/submissions/CIK{cik.zfill(10)}.json'
        headers = {'User-Agent': 'Your Name (your.email@example.com)'}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            filings = response.json().get('filings', {}).get('recent', {})
            dates = []
            for form, date in zip(filings.get('form', []), filings.get('filingDate', [])):
                if form in ['10-K', '10-Q']:
                    dates.append(date)
            return dates
        else:
            print(f"Failed to fetch data for CIK {cik}: {response.status_code}")
            return []

    # Loop through each row in the input DataFrame
    for _, row in data.iterrows():
        cik = row['CIK']
        ticker = row.get('Ticker', None)  # Optional: Retrieve ticker if available
        filing_dates = fetch_filing_dates(cik)

        # Add each filing date as a separate row in the result DataFrame
        for filing_date in filing_dates:
            new_row = {
                'CIK': cik,
                'Ticker': ticker,
                'Date': datetime.strptime(filing_date, '%Y-%m-%d')
            }
            result_data = pd.concat([result_data, pd.DataFrame([new_row])], ignore_index=True)

    return result_data


In [98]:
def get_investor_relations_pages(data):
    def get_company_website(company_name):
        search_url = f"https://www.bing.com/search?q={company_name} official website"
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(search_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        for link in soup.find_all('a'):
            href = link.get('href')
            if href and 'http' in href:
                return href
        return None

    def find_investor_relations_page(domain):
        possible_paths = ['/investor-relations', '/investors', '/ir']
        for path in possible_paths:
            url = urljoin(domain, path)
            response = requests.head(url)
            if response.status_code == 200:
                return url
        return None

    data['Website'] = data['Name'].apply(get_company_website)
    data['Investor Relations Page'] = data['Website'].apply(lambda x: find_investor_relations_page(x) if x else None)

    return data

In [88]:
data = get_dates(tickers_short)

C:\Users\Adam Krupa\AppData\Local\Temp\ipykernel_141800\2737740090.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)
BRK.B: $BRK.B: possibly delisted; no earnings dates found


BRK.B: possibly delisted; no earnings dates found


In [81]:
data = get_cik(data)

In [89]:
data = get_cik_n_names(data)

In [ ]:
data = get_investor_relations_pages(data)

In [99]:
data = get_dates_v2(data)

C:\Users\Adam Krupa\AppData\Local\Temp\ipykernel_141800\2564668417.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_data = pd.concat([result_data, pd.DataFrame([new_row])], ignore_index=True)


In [ ]:
# Replace with your CIK and timestamp of interest
cik = '0000320193'  # Example CIK for Apple Inc.
date_with_time = '2024-10-30 20:00:00-04:00'  # Timestamp with timezone

# Parse the date with timezone information
date_time = datetime.fromisoformat(date_with_time)

# Derive the two possible dates based on the time
date_day_1 = date_time.strftime('%Y-%m-%d')
date_day_2 = (date_time + timedelta(days=1)).strftime('%Y-%m-%d')

# Ensure the CIK is 10 digits with leading zeros
cik_padded = cik.zfill(10)

# SEC EDGAR Submissions URL
url = f'https://data.sec.gov/submissions/CIK{cik_padded}.json'

# Set a custom User-Agent to avoid request blocking
headers = {'User-Agent': 'Name Surname (email@gmail.com)'}

# Fetch the submissions data
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    filings = data.get('filings', {}).get('recent', {})
    filing_dates = filings.get('filingDate', [])
    form_types = filings.get('form', [])
    document_links = filings.get('primaryDocument', [])

    # Check for filings on both dates
    print(f"Checking filings for dates: {date_day_1} and {date_day_2}")
    for date, form, doc in zip(filing_dates, form_types, document_links):
        if date in [date_day_1, date_day_2]:
            print(f'Form Type: {form}')
            print(f'Filing Date: {date}')
            print(f'Document Link: https://www.sec.gov/Archives/edgar/data/{cik_padded}/{doc}')
            print('---')
else:
    print(f'Failed to fetch data: {response.status_code}')


Checking filings for dates: 2024-10-30 and 2024-10-31
Form Type: 8-K
Filing Date: 2024-10-31
Document Link: https://www.sec.gov/Archives/edgar/data/0000320193/aapl-20241031.htm
---


In [ ]:
import requests
from datetime import datetime, timedelta

# Replace with your CIK and timestamp of interest
cik = '0000320193'  # Example CIK for Apple Inc.
date_with_time = '2024-10-30 20:00:00-04:00'  # Timestamp with timezone

# Parse the date with timezone information
date_time = datetime.fromisoformat(date_with_time)

# Extract the filing date and the next day
filing_date = date_time.strftime('%Y-%m-%d')
next_day = (date_time + timedelta(days=1)).strftime('%Y-%m-%d')

# Ensure the CIK is 10 digits with leading zeros
cik_padded = cik.zfill(10)

# SEC EDGAR Submissions URL
url = f'https://data.sec.gov/submissions/CIK{cik_padded}.json'

# Set a custom User-Agent to avoid request blocking
headers = {'User-Agent': 'Name Surname (email@gmail.com)'}

# Fetch the submissions data
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    filings = data.get('filings', {}).get('recent', {})
    filing_dates = filings.get('filingDate', [])
    form_types = filings.get('form', [])
    document_links = filings.get('primaryDocument', [])
    accession_numbers = filings.get('accessionNumber', [])

    # Debugging: Print the lists
    print("Filing Dates:", filing_dates)
    print("Form Types:", form_types)
    print("Document Links:", document_links)
    print("Accession Numbers:", accession_numbers)

    # Check for filings on the specified date and the next day
    print(f"Checking filings for dates: {filing_date} and {next_day}")
    for date, form, doc, accession in zip(filing_dates, form_types, document_links, accession_numbers):
        if date in [filing_date, next_day]:
            # Format the Accession Number by removing dashes
            accession_formatted = accession.replace('-', '')
            # Construct the document link
            document_url = f'https://www.sec.gov/Archives/edgar/data/{cik_padded}/{accession_formatted}/{doc}'
            print(f'Form Type: {form}')
            print(f'Filing Date: {date}')
            print(f'Document Link: {document_url}')
            print('---')
    
    if not any(date in [filing_date, next_day] for date in filing_dates):
        print(f"No filings found for the dates: {filing_date} or {next_day}")
else:
    print(f'Failed to fetch data: {response.status_code}')


Filing Dates: ['2024-12-18', '2024-12-16', '2024-11-19', '2024-11-19', '2024-11-19', '2024-11-18', '2024-11-15', '2024-11-07', '2024-11-01', '2024-11-01', '2024-10-31', '2024-10-17', '2024-10-08', '2024-10-04', '2024-10-03', '2024-10-03', '2024-10-03', '2024-10-03', '2024-10-03', '2024-10-02', '2024-10-02', '2024-10-02', '2024-10-02', '2024-10-01', '2024-10-01', '2024-10-01', '2024-10-01', '2024-10-01', '2024-10-01', '2024-10-01', '2024-09-10', '2024-08-26', '2024-08-23', '2024-08-19', '2024-08-13', '2024-08-07', '2024-08-05', '2024-08-02', '2024-08-01', '2024-06-03', '2024-05-30', '2024-05-17', '2024-05-16', '2024-05-14', '2024-05-03', '2024-05-03', '2024-05-02', '2024-04-29', '2024-04-17', '2024-04-15', '2024-04-15', '2024-04-11', '2024-04-11', '2024-04-05', '2024-04-03', '2024-04-03', '2024-04-03', '2024-04-03', '2024-04-03', '2024-04-02', '2024-04-02', '2024-04-02', '2024-04-02', '2024-03-27', '2024-03-20', '2024-03-06', '2024-03-01', '2024-03-01', '2024-03-01', '2024-03-01', '2024

In [ ]:
import requests
from datetime import datetime, timedelta

# Replace with your CIK and timestamp of interest
cik = '0000320193'  # Example CIK for Apple Inc.
date_with_time = '2024-10-31 20:00:00-04:00'  # Timestamp with timezone

# Parse the date with timezone information
date_time = datetime.fromisoformat(date_with_time)

# Extract the filing date and the next day
filing_date = date_time.strftime('%Y-%m-%d')
next_day = (date_time + timedelta(days=1)).strftime('%Y-%m-%d')

# Ensure the CIK is 10 digits with leading zeros
cik_padded = cik.zfill(10)

# SEC EDGAR Submissions URL
url = f'https://data.sec.gov/submissions/CIK{cik_padded}.json'

# Set a custom User-Agent to avoid request blocking
headers = {'User-Agent': 'Name Surname (email@gmail.com)'}

# Fetch the submissions data
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    filings = data.get('filings', {}).get('recent', {})
    filing_dates = filings.get('filingDate', [])
    form_types = filings.get('form', [])
    document_links = filings.get('primaryDocument', [])
    accession_numbers = filings.get('accessionNumber', [])

    # Debugging: Print the lists
    print("Filing Dates:", filing_dates)
    print("Form Types:", form_types)
    print("Document Links:", document_links)
    print("Accession Numbers:", accession_numbers)

    # Check for filings on the specified date and the next day
    print(f"Checking filings for dates: {filing_date} and {next_day}")
    for date, form, doc, accession in zip(filing_dates, form_types, document_links, accession_numbers):
        if date in [filing_date, next_day] and form in ['10-K', '10-Q']:
            # Format the Accession Number by removing dashes
            accession_formatted = accession.replace('-', '')
            # Construct the document link
            document_url = f'https://www.sec.gov/Archives/edgar/data/{cik_padded}/{accession_formatted}/{doc}'
            print(f'Form Type: {form}')
            print(f'Filing Date: {date}')
            print(f'Document Link: {document_url}')
            print('---')
    
    if not any(date in [filing_date, next_day] and form in ['10-K', '10-Q'] for date, form in zip(filing_dates, form_types)):
        print(f"No filings found for the dates: {filing_date} or {next_day} matching 10-K or 10-Q")
else:
    print(f'Failed to fetch data: {response.status_code}')


Filing Dates: ['2024-12-18', '2024-12-16', '2024-11-19', '2024-11-19', '2024-11-19', '2024-11-18', '2024-11-15', '2024-11-07', '2024-11-01', '2024-11-01', '2024-10-31', '2024-10-17', '2024-10-08', '2024-10-04', '2024-10-03', '2024-10-03', '2024-10-03', '2024-10-03', '2024-10-03', '2024-10-02', '2024-10-02', '2024-10-02', '2024-10-02', '2024-10-01', '2024-10-01', '2024-10-01', '2024-10-01', '2024-10-01', '2024-10-01', '2024-10-01', '2024-09-10', '2024-08-26', '2024-08-23', '2024-08-19', '2024-08-13', '2024-08-07', '2024-08-05', '2024-08-02', '2024-08-01', '2024-06-03', '2024-05-30', '2024-05-17', '2024-05-16', '2024-05-14', '2024-05-03', '2024-05-03', '2024-05-02', '2024-04-29', '2024-04-17', '2024-04-15', '2024-04-15', '2024-04-11', '2024-04-11', '2024-04-05', '2024-04-03', '2024-04-03', '2024-04-03', '2024-04-03', '2024-04-03', '2024-04-02', '2024-04-02', '2024-04-02', '2024-04-02', '2024-03-27', '2024-03-20', '2024-03-06', '2024-03-01', '2024-03-01', '2024-03-01', '2024-03-01', '2024

In [100]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(data)

             CIK Ticker       Date
0     0000320193   AAPL 2024-11-01
1     0000320193   AAPL 2024-08-02
2     0000320193   AAPL 2024-05-03
3     0000320193   AAPL 2024-02-02
4     0000320193   AAPL 2023-11-03
5     0000320193   AAPL 2023-08-04
6     0000320193   AAPL 2023-05-05
7     0000320193   AAPL 2023-02-03
8     0000320193   AAPL 2022-10-28
9     0000320193   AAPL 2022-07-29
10    0000320193   AAPL 2022-04-29
11    0000320193   AAPL 2022-01-28
12    0000320193   AAPL 2021-10-29
13    0000320193   AAPL 2021-07-28
14    0000320193   AAPL 2021-04-29
15    0000320193   AAPL 2021-01-28
16    0000320193   AAPL 2020-10-30
17    0000320193   AAPL 2020-07-31
18    0000320193   AAPL 2020-05-01
19    0000320193   AAPL 2020-01-29
20    0000320193   AAPL 2019-10-31
21    0000320193   AAPL 2019-07-31
22    0000320193   AAPL 2019-05-01
23    0000320193   AAPL 2019-01-30
24    0000320193   AAPL 2018-11-05
25    0000320193   AAPL 2018-08-01
26    0000320193   AAPL 2018-05-02
27    0000320193   A